In [1]:
import sys, os
import requests
import json

import geopandas as gpd
import pandas as pd

from shapely.geometry import Point, Polygon, box, shape

sys.path.append("../src")

from wb_gad_helper import *

%load_ext autoreload
%autoreload 2  

In [2]:
admin_folder = r'C:\WBG\Work\data\ADMIN\NEW_WB_BOUNDS\FOR_PUBLICATION'
out_folder = r'C:\WBG\Work\data\ADMIN\REGIONAL_DISSOLVES'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

data360_group = '38.0'
data360_regions = '1.0'

adm0_file = os.path.join(admin_folder, "gpkg", "WB_GAD_ADM0_complete.gpkg")
inA = gpd.read_file(adm0_file)
inA.head()

,ISO_A3,ISO_A2,WB_A3,HASC_0,GAUL_0,WB_STATUS,SOVEREIGN,NAM_0,CONTINENT,WB_REGION,WB_INCOME,geometry
0,CHN,CN,CHN,CN,147295,Member State,CHN,China,Asia,EAS,UMC,"MULTIPOLYGON (((117.58675 38.59517, 117.58909 ..."
1,JPN,JP,JPN,JP,126,Member State,JPN,Japan,Asia,EAS,HIC,"MULTIPOLYGON (((137.48411 34.67386, 137.46683 ..."
2,KOR,KR,KOR,KR,202,Member State,KOR,Republic of Korea,Asia,EAS,HIC,"MULTIPOLYGON (((126.05363 36.19852, 126.05372 ..."
3,PRK,KP,PRK,KP,67,Non Member State,PRK,D. P. R. of Korea,Asia,EAS,LIC,"MULTIPOLYGON (((126.95508 38.16282, 126.95184 ..."
4,RUS,RU,RUS,RU,204,Member State,RUS,Russian Federation,Europe,ECS,UMC,"MULTIPOLYGON (((130.61904 48.88019, 130.60659 ..."


In [3]:
for data360_group in ['37.0', '38.0']:
    for data360_region in ['1.0','2.0']:        
        wb_classes = get_wb_classifications_strict(grouping_version=data360_group, region_version=data360_region, income_version="2.0",)
        wb_classes.rename({'ISO3':'ISO_A3', 'REGION':'WB_REGION', 'INCOME':'WB_INCOME'}, inplace=True, axis=1)
        wb_classes['CONTINENT'] = wb_classes['CONTINENT'].fillna("001")
        continent_names_map = {
            '002':'Africa',
            '005':'South America',
            '009':'Oceania',
            '142':'Asia',
            '150':'Europe',
            '001':'North America'
        }
        wb_classes['CONTINENT'] = wb_classes['CONTINENT'].map(continent_names_map)
        
        inA.drop(["WB_REGION", 'WB_INCOME', 'CONTINENT'], axis=1, inplace=True)
        inA = pd.merge(inA, wb_classes, on='ISO_A3')
        for cCol in ['CONTINENT', 'WB_REGION']:
            cur_res = []
            out_file = os.path.join(out_folder, f'WB_{cCol}_{data360_group}_{data360_region}.gpkg')
            for lbl, inD in inA.groupby(cCol):
                cur_res.append([lbl, inD.dissolve()['geometry'].iloc[0]])
            out_res = gpd.GeoDataFrame(pd.DataFrame(cur_res, columns=[cCol, 'geometry']), geometry='geometry', crs=4326)
            out_res.to_file(out_file, driver='GPKG')

In [4]:
out_file

'C:\\WBG\\Work\\data\\ADMIN\\REGIONAL_DISSOLVES\\WB_WB_REGION_38.0_2.0.gpkg'